Andrew Marasco \
*Automated Dialogue Summarization for Messaging Platform* \
Flatiron School Capstone Project #2 \
January, 2026

NOTE: This notebook was developed and trained using Google Colab GPU for reproducibility.




Environment: Google Colab (GPU)
Core model: BERT encoder + GPT-2 decoder (EncoderDecoderModel)
Dataset subset size:
Train: ~1,000–2,000 examples
Validation: ~200
Test: ~200
Max lengths:
Dialogue: 256–512 tokens
Summary: ~64 tokens

### Status 1/28/26
Step 1 Complete \
Step 2.1 complete

In [2]:
import torch
torch.cuda.is_available()

True

## Step 1: Dataset Exploration and Preparation

1.1: Loading Dataset + Inspecting Structure

In [3]:
!pip -q install -U transformers datasets evaluate accelerate rouge_score sentencepiece

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 162.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 20.5 MB/s eta 0:00:00


In [4]:
import datasets, huggingface_hub
print("datasets:", datasets.__version__)
print("huggingface_hub:", huggingface_hub.__version__)


datasets: 4.5.0
huggingface_hub: 1.3.4


In [5]:
import random
import numpy as np
import pandas as pd
import torch

from datasets import load_dataset
import evaluate

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
dataset = load_dataset("knkarthick/samsum")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})


In [7]:
print({k: len(v) for k, v in dataset.items()})
print("Columns:", dataset["train"].column_names)

{'train': 14731, 'validation': 818, 'test': 819}
Columns: ['id', 'dialogue', 'summary']


1.2: Inspecting a few examples

In [8]:
def show_example(split="train", idx=None):
    import random
    if idx is None:
        idx = random.randint(0, len(dataset[split]) - 1)
    ex = dataset[split][idx]
    print(f"Split: {split} | Index: {idx}")
    print("\n--- DIALOGUE ---")
    print(ex["dialogue"])
    print("\n--- SUMMARY (target) ---")
    print(ex["summary"])
    return ex

_ = show_example("train")
_ = show_example("train")
_ = show_example("validation")


Split: train | Index: 10476

--- DIALOGUE ---
David: The new movie of Jonhy English has come out, have you seen it?
Patricia: No but I have been meaning to go tough. I heard it's hilarious.
David: Rowan Atkison is just awesome, love that guy! In Mr. Bean I would just laugh so hard ahaha
Patricia: Me too 😂 I couldn't watch some scenes sometimes cause they would make me nervous from all the constant crap he did ahhaha
David: ahahaa xD  Anyway.. wanna go to the 21:40 session today? I ain't got much going on so..
Patricia: Sure! Where are you having dinner?
David: Was thinking of just ordering a pizza, you have any ideas?
Patricia: There's a new Mexican place and they do take out's, want me to grab something and meet you at your place?
David: Oh that's what I'm talking about! Bring me 2 chicken burritos and nachoooos with guacamole.
Patricia: Anything else for the little boy? ahaha xD
David: While you're at it a coke would do 😂
Patricia: Jesus.. x) Leaving my place now, cya in a bit.

--- 

1.3: Analyzing Characteristics (distribution of length)

In [9]:
def length_stats(split="train", n=2000):
  n = min(n, len(dataset[split]))
  sample = dataset[split].select(range(n))
  df = pd.DataFrame({
      "dialogue_words": [len(x.split()) for x in sample["dialogue"]],
      "summary_words": [len(x.split()) for x in sample["summary"]],
  })
  return df.describe(percentiles=[.5, .8, .9, .95, .99])

stats = length_stats("train", n=2000)
stats

,dialogue_words,summary_words
count,2000.000000,2000.000000
mean,95.368500,20.521000
std,73.369355,11.365524
min,7.000000,1.000000
50%,75.000000,18.000000
80%,142.000000,30.000000
90%,194.000000,37.000000
95%,248.050000,44.000000
99%,344.060000,53.000000
max,471.000000,60.000000


1.4: Creating Training/Validation Splits for Project
Note: Dataset is already split, but this step exists in the project instructions

In [10]:
SEED = 42
TRAIN_SIZE = 2000
VAL_SIZE = 300

train_ds = dataset["train"].shuffle(seed=SEED).select(range(TRAIN_SIZE))
val_ds   = dataset["validation"].shuffle(seed=SEED).select(range(VAL_SIZE))

print(len(train_ds), len(val_ds))

2000 300


1.5: Tokenization Setup using BERT encoder and GPT-2 Decoder

In [11]:
from transformers import AutoTokenizer

encoder_name = "bert-base-uncased"
decoder_name = "gpt2"

enc_tok = AutoTokenizer.from_pretrained(encoder_name)
dec_tok = AutoTokenizer.from_pretrained(decoder_name)

# Setting GPT-2 pad token to EOS
dec_tok.pad_token = dec_tok.eos_token

MAX_INPUT_LEN = 512
MAX_TARGET_LEN = 64

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
def preprocess_batch(batch):
  # encode dialogue (encoder input)
  enc = enc_tok(
      batch["dialogue"],
      truncation=True,
      padding="max_length",
      max_length=MAX_INPUT_LEN,
  )

  # encoding summary (decoder labels)
  dec = dec_tok(
      batch["summary"],
      truncation=True,
      padding="max_length",
      max_length=MAX_TARGET_LEN,
  )

  enc["labels"] = dec["input_ids"]
  return enc

In [13]:
train_tok = train_ds.map(preprocess_batch, batched=True, remove_columns=train_ds.column_names)
val_tok = val_ds.map(preprocess_batch, batched=True, remove_columns=val_ds.column_names)

train_tok.set_format(type="torch")
val_tok.set_format(type="torch")

train_tok[0]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

{'input_ids': tensor([  101,  4205,  1024,  1026,  5371,  1035,  2678,  1028,  4205,  1024,
          2054,  2079,  2017,  2228, 10590,  1024,  2507,  2033,  1037, 10819,
         10590,  1024,  7929,  3666,  4205,  1024,  2292,  2033,  2113, 10590,
          1024,  2064,  1005,  1056,  2428,  2963,  1037,  2843,  2045,  1025,
          1013,  4205,  1024,  3398,  1025,  1013,  4205,  1024,  1045,  2228,
          1045,  2342,  2000,  2501,  2009,  5064,  2842,  4205,  1024,  2672,
          2083,  1996,  8278,  1998,  4007, 10590,  1024,  2008,  5791,  2003,
          1037,  2307,  2801,   999, 10590,  1024,  1045,  3984,  2008,  1005,
          1055,  2339,  1045,  2435,  2017,  1996,  8278,  1998,  5361,  2009,
          1024,  1040,  4205,  1024,  3398,  1060,  2094,  4205,  1024,  7929,
          1045,  1005,  2222,  3046,  2000,  3275,  2009,  2041,  2101, 10590,
          1024,  7929, 10590,  1024,  1045,  1005,  2222,  2022,  3403,  1024,
          1052,   102,     0,     0,   

1.6: Building DataLoaders for efficient model training

In [14]:
from torch.utils.data import DataLoader

BATCH_SIZE = 8

train_loader = DataLoader(train_tok, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_tok, batch_size=BATCH_SIZE, shuffle=False)

batch = next(iter(train_loader))
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 512]),
 'token_type_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512]),
 'labels': torch.Size([8, 64])}

## Step 2: Model Architecture Implementation

2.1: Creating the Encoder-Decoder Model (BERT -> GPT2)

In [15]:
from transformers import EncoderDecoderModel

encoder_name = "bert-base-uncased"
decoder_name = "gpt2"

model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_name,
    decoder_name
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                                                 | Status     | 
----------------------------------------------------+------------+-
h.{0...11}.attn.bias                                | UNEXPECTED | 
transformer.h.{0...11}.crossattention.c_proj.bias   | MISSING    | 
transformer.h.{0...11}.crossattention.c_proj.weight | MISSING    | 
transformer.h.{0...11}.ln_cross_attn.bias           | MISSING    | 
transformer.h.{0...11}.ln_cross_attn.weight         | MISSING    | 
transformer.h.{0...11}.crossattention.q_attn.bias   | MISSING    | 
transformer.h.{0...11}.crossattention.q_attn.weight | MISSING    | 
transformer.h.{0...11}.crossattention.c_attn.bias   | MISSING    | 
transformer.h.{0...11}.crossattention.c_attn.weight | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

2.2: Configuring Special Tokens (EOS/PAD/start tokens)

In [17]:
# decoder (GPT2) special token IDs
model.config.eos_token_id = dec_tok.eos_token_id
model.config.pad_token_id = dec_tok.pad_token_id

# Start token for decoding:
model.config.decoder_start_token_id = dec_tok.bos_token_id or dec_tok.eos_token_id

2.3: Setting generation defaults (beam search, length, etc.)

In [18]:
from transformers import GenerationConfig

model.generation_config = GenerationConfig(
    max_new_tokens=64,
    num_beams=4,
    early_stopping=True,
    length_penalty=
)

2.4: Move Model to GPU

In [20]:
model = model.to(device)
device

'cuda'

2.5: Forward Pass Check

In [21]:
batch = next(iter(train_loader))
batch = {k: v.to(device) for k, v in batch.items()}

out = model(**batch)
print("Loss:", float(out.loss))
print("Logits shape:", tuple(out.logits.shape))


Loss: 7.537334442138672
Logits shape: (8, 64, 50257)


/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:445: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/tmp/ipython-input-3024014686.py:5: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  print("Loss:", float(out.loss))


2.6: Building Simple Inference Function (Prototype 1)

In [28]:
def generate_summary(dialogue: str, num_beams=4, max_new_tokens=64):
  inputs = enc_tok(
      dialogue,
      return_tensors="pt",
      truncation=True,
      padding="max_length",
      max_length=MAX_INPUT_LEN
  ).to(device)

  summary_ids = model.generate(
      input_ids=inputs["input_ids"],
      attention_mask=inputs["attention_mask"],
      num_beams=num_beams,
      max_new_tokens=max_new_tokens,
      early_stopping=True
  )

  return dec_tok.decode(summary_ids[0], skip_special_tokens=True)

Testing on real sample

In [29]:
ex = dataset["validation"][0]
print(ex["dialogue"])
print("\n---model output (untrained) ---")
print(generate_summary(ex["dialogue"]))
print("\n--- target summary ---")
print(ex["summary"])

A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))

---model output (untrained) ---


ValueError: You have modified the pretrained model configuration to control generation We detected the following values set - {'max_length': 64, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 1.0}. This strategy to control generation is not supported anymore. Please use and modify `model.generation_config` (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )